In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import sys 
sys.path.append('/mnt/ceph_rbd/LoRa/student_llm_kt/src')
sys.path.append('/mnt/ceph_rbd/LoRa/student_llm_kt/src/DKT_src')
from training import train_dynamic_kt
from LoRa_preprocessing import StudentInteractionsDataset, DataFrame2InteractionDictionary,load_data
from dataloader_new import SequenceDataset, options_dataloader_preproc_process
answers,questions,misconceptions,question_subject = load_data('/mnt/ceph_rbd/LoRa/data')
from torch.utils.data import DataLoader



In [ ]:
data = DataFrame2InteractionDictionary(answers,questions,misconceptions,question_subject,train_split=0.9)
data.createedi(3456784,'/mnt/ceph_rbd/LoRa/filtered_interaction_dictionaries/eedi_train0.9.pt')

In [ ]:
data.train_dictionary

In [ ]:
data = SequenceDataset(data.train_dictionary)

In [ ]:
data.data.keys()

In [ ]:
    # Create dataloaders
train_loader = DataLoader(
    SequenceDataset(data.train_dictionary),
    batch_size=9,
    shuffle=True,
    collate_fn=options_dataloader_preproc_process,
    num_workers=2
)

In [ ]:
first_batch = next(iter(train_loader))


In [ ]:
first_batch

In [ ]:
test = data.train_dictionary

first_10_dict = {k: test[k] for k in list(test.keys())[:len(list(test.keys()))*0.1]}

In [ ]:
# Create data loaders
train_loader = DataLoader(
    SequenceDataset(test), 
    batch_size=5, 
    drop_last=True, 
    shuffle=True, 
    collate_fn=options_dataloader_preproc_process, 
    num_workers=10
)

In [ ]:
first_batch = next(iter(train_loader))

In [ ]:
first_batch['question_ids'].shape

In [ ]:
first_batch['QuestionEmbedding'].shape

In [ ]:
feature_config = {
    'question_id': {
        'type': 'embedding',
        'num_embeddings': 10000,
        'embedding_dim': 64
    },
    'options': {
        'type': 'embedding',
        'num_embeddings': 4,
        'embedding_dim': 23
    },
    'misconceptions': {
        'type': 'embedding_with_mask',
        'num_embeddings': 100000,
        'embedding_dim': 65,
        'input_key': 'misconceptions',
        'mask_key': 'misconception_mask'
    }
}

model_config = {
    'hidden_dim': 256,
    'nhead': 2,
    'num_layers': 1,
    'dropout': 0.1
}

training_config = {
    'batch_size': 32,
    'max_epochs': 100,
    'patience': 10,
    'learning_rate': 0.001,
    'save_model': True,
    'save_dir': 'models/',
    'num_workers': 4
}

best_model_state, best_metrics = train_dynamic_kt(
    train_data=train_loader,
    val_data=train_loader,
    feature_config=feature_config,
    model_config=model_config,
    training_config=training_config,
    experiment_name="DKT_experiment"
)